# Mobile Metrics

Loads and saves webpagetest.org performance tests.

Needs a local installation of the webpagetest tools https://github.com/WPO-Foundation/webpagetest and a mobile device or a key for the webpagetest API.

A complete test may last for up to 2 hours.

Feel free to contact me for help: https://www.quel-media.com/about.html#contact

© Paul Ronga under Apache-2 Licence (see LICENCE.txt).

In [1]:
import pandas as pd
import requests
from IPython.display import HTML
import json
import datetime

In [2]:
# Link to local PHP files
RUN_TEST_URL = 'http://rospo.local/~paul/webpagetest/run_mobile.php'
GET_TEST_URL = 'http://rospo.local/~paul/webpagetest/get.php?test_id={}'

In [3]:
# dataframe containing media id, name and URLs
medias = pd.read_csv('df/media_list.csv')

# remove Konbini ch and fr
medias = medias[medias['media_id'] < 34].copy()

# media id as string
medias['media_id'] = medias['media_id'].apply(lambda x: str(x))

medias.head(2)

,media_id,Name,URL_short,URL,URL_mobile
0,19,La Tribune de Genève,tdg.ch,https://www.tdg.ch/,https://m.tdg.ch
1,20,24 heures,24heures.ch,https://www.24heures.ch,https://m.24heures.ch


# 1. Lauch tests

Lauch tests (average of 3 runs) for each media on a device monitored with the webpagetest system.

In [4]:
reports = {}

In [6]:
for i, row in medias.iterrows():
    url = row['URL_mobile']
    if url != url or url == 'x':
        url = row['URL']

    print('Running test for', url)

    payload = {'url': url}
    r = requests.post(RUN_TEST_URL, json=payload)
    
    print(r.text, end='\n\n')
    result = r.json()
    reports[row['media_id']] = result['id']

Running test for https://m.tdg.ch
{"id":"180803_J5_b405d7b7f65bbbc3d5bec60c22beae95", "url":"https://m.tdg.ch"}

Running test for https://m.24heures.ch
{"id":"180803_YW_0dc3146750b11066361292cacd2066c3", "url":"https://m.24heures.ch"}

Running test for https://www.letemps.ch
{"id":"180803_4P_9b550833fd5cb35e496f775e05359d08", "url":"https://www.letemps.ch"}

Running test for https://mobile.lemonde.fr
{"id":"180803_FV_18959e95c3f4f5fbd5acc348c3d17692", "url":"https://mobile.lemonde.fr"}

Running test for https://www.rts.ch/info/
{"id":"180803_P2_1cbd09a3cd92e7931b8292fd6c9afc35", "url":"https://www.rts.ch/info/"}

Running test for https://m.20min.ch/ro/
{"id":"180803_1S_a3b17e1da8bb3b98276b3402764d094c", "url":"https://m.20min.ch/ro/"}

Running test for https://m.lematin.ch
{"id":"180803_87_ff0a9faed4f9b3837acb24f89a04b3c0", "url":"https://m.lematin.ch"}

Running test for https://www.mediapart.fr
{"id":"180803_QT_0b99f18ffd3d627eeb46415055cc7543", "url":"https://www.mediapart.fr"}

Runn

In [8]:
reports

{'19': '180803_J5_b405d7b7f65bbbc3d5bec60c22beae95',
 '20': '180803_YW_0dc3146750b11066361292cacd2066c3',
 '21': '180803_4P_9b550833fd5cb35e496f775e05359d08',
 '22': '180803_FV_18959e95c3f4f5fbd5acc348c3d17692',
 '23': '180803_P2_1cbd09a3cd92e7931b8292fd6c9afc35',
 '24': '180803_1S_a3b17e1da8bb3b98276b3402764d094c',
 '25': '180803_87_ff0a9faed4f9b3837acb24f89a04b3c0',
 '26': '180803_QT_0b99f18ffd3d627eeb46415055cc7543',
 '27': '180803_43_7ba6bc702c145fdfe413b16e96fd9490',
 '28': '180803_Y1_73b57953e356bca385e10b3194ea6f8e',
 '29': '180803_QX_af12c1db77dd2998e019638172ee4c0d',
 '30': '180803_VM_dfff6d57f31f0c9396c8eb767029d5ba',
 '31': '180803_Q9_98de2bdd5086607eb32c354af2cc26ed',
 '32': '180803_WF_53cc9a5c325b38cbf82fc0ad1bfaf29e',
 '33': '180803_PR_c949d950a2d29183dd8af302974d5d32'}

# 2. Get test results

Wait about one hour before trying to get these results.

In [11]:
df_speed = pd.DataFrame(columns=['Name', 'media_id', 'location', 'visualComplete', 'fullyLoaded', 'requests', 'report_url', 'completed'])

In [12]:
from IPython.display import HTML
#HTML(r.text)

In [14]:
for i, row in medias.iterrows():
    print('Getting results for', row['Name'], '...')
    
    if len(df_speed[df_speed['media_id'] == row['media_id']]) > 0:
        print('Result already fetched.')
    else:    
        r = requests.get(GET_TEST_URL.format( reports[row['media_id']] ))
        print(r)
        try:
            data = r.json()
        except ValueError as e:
            print(e)
            print(r.text)

        missing_key = False

        if 'data' in data.keys() and 'average' in data['data'].keys():    
            print('Successful first view runs:', data['data']['successfulFVRuns'])

            print('Location and device were:', data['data']['location'], 'with connectivity:', data['data']['connectivity'])
            
            for key in ['visualComplete', 'fullyLoaded', 'requests']:
                if key not in data['data']['average']['firstView'].keys():
                    print('Missing key:', key)
                    print(data['data']['average']['firstView'].keys())
                    if key == 'visualComplete' and 'fullyLoaded' in data['data']['average']['firstView'].keys():
                        data['data']['average']['firstView']['visualComplete'] = data['data']['average']['firstView']['fullyLoaded']
                    else:
                        missing_key = True

            if not missing_key:
                df_speed = df_speed.append(pd.DataFrame([[
                    row['Name'],
                    row['media_id'],
                    data['data']['location'],
                    int(data['data']['average']['firstView']['visualComplete']),
                    int(data['data']['average']['firstView']['fullyLoaded']),
                    int(data['data']['average']['firstView']['requests']),
                    data['data']['summary'],
                    data['data']['completed']
                ]], columns=['Name', 'media_id', 'location', 'visualComplete', 'fullyLoaded', 'requests', 'report_url', 'completed']))
            
                with open('df/archive/mobile/{}.json'.format(data['data']['testId']), 'w') as outfile:
                    json.dump(data['data'], outfile)
                print('Saved to disk and json stored in df/archive/mobile/', end='\n\n')
        else:
            print('No result for {} yet.'.format(row['Name']))

Getting results for La Tribune de Genève ...
Result already fetched.
Getting results for 24 heures ...
Result already fetched.
Getting results for Le Temps ...
Result already fetched.
Getting results for Le Monde ...
Result already fetched.
Getting results for RTS info ...
Result already fetched.
Getting results for 20 minutes (ch) ...
Result already fetched.
Getting results for Le Matin ...
Result already fetched.
Getting results for Mediapart ...
Result already fetched.
Getting results for Le Figaro ...
Result already fetched.
Getting results for Libération ...
Result already fetched.
Getting results for La Côte ...
<Response [200]>
Successful first view runs: 4
Location and device were: Dulles_iPhone8:iPhone 8 iOS 11 with connectivity: 4G
Saved to disk and json stored in df/archive/mobile/

Getting results for Arcinfo ...
<Response [200]>
Successful first view runs: 4
Location and device were: Dulles_iPhone8:iPhone 8 iOS 11 with connectivity: 4G
Saved to disk and json stored in df/a

In [15]:
df_speed.sort_values('fullyLoaded')
#df_speed.sort_values('visualComplete')

,Name,media_id,location,visualComplete,fullyLoaded,requests,report_url,completed
0,Le Monde,22,Dulles_iPhone8:iPhone 8 iOS 11,4521,9221,85,http://www.webpagetest.org/result/180803_FV_18...,"Fri, 03 Aug 2018 09:06:33 +0000"
0,RTS info,23,Dulles_iPhone8:iPhone 8 iOS 11,8696,11604,160,http://www.webpagetest.org/result/180803_P2_1c...,"Fri, 03 Aug 2018 09:10:15 +0000"
0,Le Temps,21,Dulles_iPhone8:iPhone 8 iOS 11,7817,15492,158,http://www.webpagetest.org/result/180803_4P_9b...,"Fri, 03 Aug 2018 09:03:09 +0000"
0,Le Matin,25,Dulles_iPhone8:iPhone 8 iOS 11,13747,16356,207,http://www.webpagetest.org/result/180803_87_ff...,"Fri, 03 Aug 2018 09:26:36 +0000"
0,Le Courrier,33,Dulles_iPhone8:iPhone 8 iOS 11,12105,16881,70,http://www.webpagetest.org/result/180803_PR_c9...,"Fri, 03 Aug 2018 10:13:23 +0000"
0,20 minutes (ch),24,Dulles_iPhone8:iPhone 8 iOS 11,8793,17181,105,http://www.webpagetest.org/result/180803_1S_a3...,"Fri, 03 Aug 2018 09:18:20 +0000"
0,Le Nouvelliste,31,Dulles_iPhone8:iPhone 8 iOS 11,11775,18305,147,http://www.webpagetest.org/result/180803_Q9_98...,"Fri, 03 Aug 2018 10:03:50 +0000"
0,La Tribune de Genève,19,Dulles_iPhone8:iPhone 8 iOS 11,20614,21160,182,http://www.webpagetest.org/result/180803_J5_b4...,"Fri, 03 Aug 2018 08:52:41 +0000"
0,Mediapart,26,Dulles_iPhone8:iPhone 8 iOS 11,17001,22166,99,http://www.webpagetest.org/result/180803_QT_0b...,"Fri, 03 Aug 2018 09:31:03 +0000"
0,24 heures,20,Dulles_iPhone8:iPhone 8 iOS 11,19193,23183,276,http://www.webpagetest.org/result/180803_YW_0d...,"Fri, 03 Aug 2018 08:58:49 +0000"


In [16]:
df_speed['mobile_index'] = df_speed['fullyLoaded'].apply(lambda x: round(100 - (int(x)-5000)/1000))
df_speed.sort_values('mobile_index')

,Name,media_id,location,visualComplete,fullyLoaded,requests,report_url,completed,mobile_index
0,Le Figaro,27,Dulles_iPhone8:iPhone 8 iOS 11,13001,42903,290,http://www.webpagetest.org/result/180803_43_7b...,"Fri, 03 Aug 2018 09:40:58 +0000",62
0,La Côte,29,Dulles_iPhone8:iPhone 8 iOS 11,11946,31231,170,http://www.webpagetest.org/result/180803_QX_af...,"Fri, 03 Aug 2018 09:53:47 +0000",74
0,Libération,28,Dulles_iPhone8:iPhone 8 iOS 11,4172,30328,812,http://www.webpagetest.org/result/180803_Y1_73...,"Fri, 03 Aug 2018 09:46:48 +0000",75
0,La Liberté,32,Dulles_iPhone8:iPhone 8 iOS 11,24101,28739,147,http://www.webpagetest.org/result/180803_WF_53...,"Fri, 03 Aug 2018 10:09:12 +0000",76
0,Arcinfo,30,Dulles_iPhone8:iPhone 8 iOS 11,7105,27085,138,http://www.webpagetest.org/result/180803_VM_df...,"Fri, 03 Aug 2018 09:58:41 +0000",78
0,24 heures,20,Dulles_iPhone8:iPhone 8 iOS 11,19193,23183,276,http://www.webpagetest.org/result/180803_YW_0d...,"Fri, 03 Aug 2018 08:58:49 +0000",82
0,Mediapart,26,Dulles_iPhone8:iPhone 8 iOS 11,17001,22166,99,http://www.webpagetest.org/result/180803_QT_0b...,"Fri, 03 Aug 2018 09:31:03 +0000",83
0,La Tribune de Genève,19,Dulles_iPhone8:iPhone 8 iOS 11,20614,21160,182,http://www.webpagetest.org/result/180803_J5_b4...,"Fri, 03 Aug 2018 08:52:41 +0000",84
0,Le Nouvelliste,31,Dulles_iPhone8:iPhone 8 iOS 11,11775,18305,147,http://www.webpagetest.org/result/180803_Q9_98...,"Fri, 03 Aug 2018 10:03:50 +0000",87
0,20 minutes (ch),24,Dulles_iPhone8:iPhone 8 iOS 11,8793,17181,105,http://www.webpagetest.org/result/180803_1S_a3...,"Fri, 03 Aug 2018 09:18:20 +0000",88


In [17]:
outputfile = 'df/archive/mobile_metrics_{}.csv'.format( datetime.datetime.now().strftime('%Y-%m-%d') )
print('Saving to {}...'.format(outputfile))

Saving to df/archive/mobile_metrics_2018-08-03.csv...


In [18]:
df_speed.to_csv(outputfile) # archive
df_speed.to_csv('df/mobile_metrics.csv') # temp file

### stop here

In [15]:
df = pd.read_csv('/Users/paul/Sites/d3_v5/indices/data/media_metrics_2018-06-17.csv')

In [16]:
df_mobile = pd.read_csv('df/mobile_metrics.csv', usecols=['media_id', 'location', 'visualComplete', 'fullyLoaded',
       'requests', 'report_url', 'mobile_index'])

In [17]:
df.merge(df_mobile, on='media_id').to_csv('/Users/paul/Sites/d3_v5/indices/data/media_metrics_2018-06-17.csv')

### NB

** This script only uses loading time. Optimization is evaluated with PageSpeed. But these 9 scores are also available: **
```'score_cache',
'score_cdn',
'score_combine',
'score_compress',
'score_cookies',
'score_etags',
'score_gzip',
'score_keep-alive',
'score_minify'```